# Week Three Assignment - Question 3

## Steps followed in creating the required map

* Install all the libraries
* Download all the modules
* Read data from the Wikipedia Page
* Creating BeautifulSoup object 
* Reading and coverting BeautifulSoup oject data into dataframe
* Cleaning the data frame to remove rows with no Borough name
* Concatenating Neighborhood names which have the same Postal Code
* Adding Neighborhood names for rows missing the same
* Verify final output is corrrect and also print the shape of the final dataframe
* Reading the csv file with latitude and longitude value
* Modify column names inorder to perform the merge
* Merging data to create the new data set with latitude and longitude data
* Create a data frame with Neighborhoods from Boroughs that have the word 'Toronto' in it
* Get the Latitude and Longitude values for Toronoto
* Create a map of Toronto using latitude and longitude values and marking all the neighborhood
* Initialize Foursquare Credentials
* Create a function to retrive venue information from Forsquare for Toronto
* Create and store the venues information returned by the user defined funtion
* Performe one hot encoding for the venues data frame and rearranging the resulting data frame for further processing
* Group the data frame created using one hot coding based on the Neighborhood
* Create a function for returning the top most 'n' common venues 
* Initialize the number of top most common venues we want to explore
* Create a data frame to store the top most common venues
* Perform Clustering
* Set number of clusters you want to create and then cluster the data frame based on the same
* Create a merged data frame using the data from 'grouped by venue' and the 'main toronto data' frame to get the latitude and logitude value for each neighborhood
* Assign the cluster information to the merged data frame
* Create map with the custer information from the merged data frame and add markers to the map

In [ ]:
# intalling beaitifulsoup library
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge requests --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

In [129]:
# importing necessary modules
import requests
import pandas as pd
import numpy as np
import lxml
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 

from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

In [130]:
# url for the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

In [131]:
# cretaing a beautifulsoup obect
soup = BeautifulSoup(source,'lxml')

In [132]:
# creating & loading data into dataframe
pcode_df = pd.DataFrame()
i = 0
j = 0
table_pcode = soup.find('table')
for table_data in table_pcode.find_all('td'):
    col_data = table_data.text
    pcode_df.loc[i,j] = col_data
    j = j + 1
    if (j==3):
        i = i + 1
        j = 0

In [133]:
# adding column names to the data frame
pcode_df.columns = ('PostalCode','Borough','Neighborhood')

In [134]:
# removing the \n haracter from the end of the string
pcode_df['Neighborhood'] = pcode_df['Neighborhood'].str.slice(stop = -1)

In [135]:
# removing the rows with borough value as not assigned
pcode_df=pcode_df[pcode_df.Borough != 'Not assigned']
pcode_df = pcode_df.reset_index(drop=True)

In [136]:
# concatenating neighborhoods with same Postal Code
new_df = pd.DataFrame()
new_df = pcode_df.groupby('PostalCode').agg({'Neighborhood':lambda x: tuple(x)})
new_df.reset_index(inplace=True)

In [137]:
# converting the tuple into a string data type
new_df['Neighborhood'] = new_df['Neighborhood'].astype(str)

In [138]:
#removing the end and begenning braces and the single quotes
new_df['Neighborhood'] = new_df['Neighborhood'].str.slice(stop = -2)
new_df['Neighborhood'] = new_df['Neighborhood'].str.slice(start = 2)
new_df['Neighborhood'] = new_df['Neighborhood'].str.replace("'","")

In [139]:
# creating a data frame of unique postal code and borough combination
df_borough = pcode_df
df_borough = df_borough.drop('Neighborhood',axis=1)
df_borough.drop_duplicates(inplace=True)

In [140]:
# creation of the final data frame by combining two intermediate data frames using Postal code as key
final_pcode_df = pd.DataFrame()
final_pcode_df = df_borough.merge(new_df, on='PostalCode')

In [141]:
# updating the data for rows missing Neighborhood values
loc_Neigh_NA = final_pcode_df.Neighborhood == 'Not assigned'
final_pcode_df.loc[loc_Neigh_NA,'Neighborhood'] = final_pcode_df.loc[loc_Neigh_NA,'Borough']
final_pcode_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [142]:
# shape of the final dataframe
print('Shape of the Final DataFrame',final_pcode_df.shape)

Shape of the Final DataFrame (103, 3)


Attempt with geocode was unsuccesful. Attempt with geopy was also unsuccesful. Finally used the csv file provided to complete the assignmentto get the latitude and logitude information

In [143]:
# reading the csv file with latitude and longitude value
lat_long_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [144]:
# modify column names inorder to perform the merge
lat_long_df.columns = ('PostalCode','Latitude','Longitude')

In [145]:
# merging data to create the new data set with latitude and longitude data
final_lat_long_df = final_pcode_df.merge(lat_long_df , on='PostalCode')
final_lat_long_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [146]:
# Creating a data frame with Neighborhoods from Boroughs that have the word 'Toronto' in it
toronto_df = final_lat_long_df[final_lat_long_df['Borough'].str.contains('Toronto', case=False)].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [147]:
# getting the Latitude and Longitude values for Toronoto
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = "my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [148]:
# creating map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [150]:
# The code was removed by Watson Studio for sharing.

In [151]:
# creating a function to retrive venue information from Forsquare 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=pd.DataFrame(columns=('Neighborhood','Neigh_Latitude','Neigh_Longitude','Venue','Venue_Latitude', \
                  'Venue_Longitude','Venue_Category'))
    j = 0
    if (venues_list.shape[0] == 0):
        i = 0
    else:
        i = venues_list.shape[0]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']   
        # store the relevant information for each nearby venue in adate frame
        for v in results:
            venues_list.loc[i,'Venue'] = v['venue']['name']
            venues_list.loc[i,'Venue_Latitude'] = v['venue']['location']['lat']
            venues_list.loc[i,'Venue_Longitude'] = v['venue']['location']['lng'] 
            venues_list.loc[i,'Venue_Category'] = v['venue']['categories'][0]['name']
            venues_list.loc[i,'Neighborhood'] = names[j]
            venues_list.loc[i,'Neigh_Latitude'] = latitudes[j]
            venues_list.loc[i,'Neigh_Longitude'] = longitudes[j]
            i = i + 1
        j = j + 1
    # return the data frame
    return(venues_list)

In [152]:
# creating and storing the venues information returned by the user defined funtion
toronto_venues = pd.DataFrame()
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )
toronto_venues.head()

,Neighborhood,Neigh_Latitude,Neigh_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,"Harbourfront, Regent Park",43.6543,-79.3606,Roselle Desserts,43.6534,-79.362,Bakery
1,"Harbourfront, Regent Park",43.6543,-79.3606,Tandem Coffee,43.6536,-79.3618,Coffee Shop
2,"Harbourfront, Regent Park",43.6543,-79.3606,Cooper Koo YMCA,43.6532,-79.3579,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.6543,-79.3606,Morning Glory Cafe,43.6539,-79.3611,Breakfast Spot
4,"Harbourfront, Regent Park",43.6543,-79.3606,Body Blitz Spa East,43.6547,-79.3599,Spa


In [153]:
#verifying the number of unique categories in the venues data frame
print('There are {} uniques categories.'.format(len(toronto_venues['Venue_Category'].unique())))

There are 234 uniques categories.


In [154]:
# performing one hot encoding for the venues data frame and rearranging the resulting data frame for further processing
toronto_onehot = pd.DataFrame()
toronto_onehot = pd.get_dummies(toronto_venues['Venue_Category'])

# add neighborhood column back to dataframe
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape

(1698, 234)

In [155]:
# Group the data frame created using one hot coding based on the Neighborhood
toronto_grouped = pd.DataFrame()
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0000
1,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0000
3,Business reply mail Processing Centre969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0625
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0000


In [111]:
# create a function for returning the top n most common venues 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [156]:
# initiaitng the number of top venues we want to explaore
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
col_name = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        col_name.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        col_name.append('{}th Most Common Venue'.format(ind+1))

# creating a data frame to store the top n most common venues
toronto_venues_sorted = pd.DataFrame(columns = col_name)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Clothing Store,Hotel,Gym,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Pub,Café,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Furniture / Home Store,Convenience Store,Climbing Gym,Caribbean Restaurant,Restaurant,Burrito Place
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boutique,Sculpture Garden,Harbor / Marina,Boat or Ferry,Plane,Airport Gate,Airport Food Court


In [157]:
# set number of clusters and cluster the venues data frame based on the same
toronto_grouped_clustering = pd.DataFrame()
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [158]:
# creating a merged dat frame using the data frame grouped by venue and the main toronto data frame to get the
# latitude and logitude value for each neighborhood
toronto_merged = pd.DataFrame()
toronto_merged = toronto_grouped.merge(toronto_df, on='Neighborhood')
# add clustering labels to the merged data frame
toronto_merged['Cluster_Labels'] = kmeans.labels_

In [159]:
# create map with the custer information from the merged data frame
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [160]:
# add markers to the map based on the clusters
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], \
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters